# Smets & Wouters (2007) Direct Replication

Este notebook replica los resultados del paper usando Dynare vía oct2py.

**Requisitos previos:**
- GNU Octave instalado y en PATH
- Dynare instalado
- Sims VARtools en `repo/`

Ver `setup_instructions.md` para detalles de instalación.

## 1. Setup y Configuración

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent.parent))

from direct_replication import (
    DynareInterface,
    SimsBVAR,
    extract_state_space_matrices,
    extract_parameter_estimates,
    ReplicationVerification
)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
# Configurar rutas - MODIFICAR SEGÚN TU INSTALACIÓN
DYNARE_PATH = r'C:\dynare\6.2\matlab'  # Ruta a carpeta matlab de Dynare
REPO_PATH = Path.cwd().parent.parent / 'repo'  # Carpeta con archivos .mod

print(f"Dynare path: {DYNARE_PATH}")
print(f"Repo path: {REPO_PATH}")
print(f"Repo exists: {REPO_PATH.exists()}")

### Test de Conexión Octave + Dynare

In [ ]:
from oct2py import Oct2Py

# Test básico
try:
    oc = Oct2Py()
    result = oc.eval('2 + 2', nout=1)
    print(f"✓ Octave conectado: 2 + 2 = {result}")
    
    # Test Dynare
    oc.addpath(DYNARE_PATH)
    dynare_exists = oc.eval('exist("dynare")', nout=1)
    if dynare_exists == 2:
        print("✓ Dynare encontrado")
    else:
        print("✗ Dynare NO encontrado - verificar DYNARE_PATH")
    
    # Test VARtools
    oc.addpath(str(REPO_PATH))
    for func in ['varprior', 'rfvar3', 'matrictint']:
        exists = oc.eval(f'exist("{func}")', nout=1)
        if exists == 2:
            print(f"✓ {func}.m encontrado")
        else:
            print(f"✗ {func}.m NO encontrado - descargar de sims.princeton.edu")
    
    oc.exit()
    print("\n¡Todo listo!")
    
except Exception as e:
    print(f"Error: {e}")
    print("Ver setup_instructions.md para solución de problemas")

## 2. Cargar Datos

In [ ]:
# Cargar datos de Smets & Wouters
data_path = REPO_PATH / 'usmodel_data.xls'

# Leer datos
data = pd.read_excel(data_path)

print(f"Datos cargados: {data.shape}")
print(f"\nVariables: {list(data.columns)}")
print(f"\nPrimeras observaciones:")
data.head()

In [ ]:
# Plot de series observables
fig, axes = plt.subplots(4, 2, figsize=(14, 10))
axes = axes.flatten()

observable_vars = ['dy', 'dc', 'dinve', 'labobs', 'pinfobs', 'dw', 'robs']

for i, var in enumerate(observable_vars):
    if var in data.columns:
        axes[i].plot(data[var])
        axes[i].set_title(var)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEstadísticas descriptivas:")
data[observable_vars].describe()

## 3. Ejecutar Dynare en usmodel.mod

In [ ]:
# Inicializar interfaz Dynare
di = DynareInterface(DYNARE_PATH, str(REPO_PATH))

print("Interfaz Dynare inicializada")

In [ ]:
# Ejecutar modelo
# Nota: Esto puede tardar varios minutos
print("Ejecutando Dynare en usmodel.mod...")
print("(Esto puede tardar 5-10 minutos)\n")

di.run_model('usmodel.mod')

print("\n✓ Dynare completado")

## 4. Extraer Resultados DSGE

### 4.1 Parámetros Estimados

In [ ]:
# Extraer parámetros
params_df = di.get_parameters()

print("Parámetros estimados (posterior mode):\n")
print(params_df.to_string())

# Guardar a CSV
params_df.to_csv('parameter_estimates.csv', index=False)
print("\n✓ Parámetros guardados en parameter_estimates.csv")

### 4.2 Matrices State-Space

In [ ]:
# Extraer state-space
ss = di.get_state_space()

print(f"Matriz T (state transition): {ss['T'].shape}")
print(f"Matriz R (shock impact): {ss['R'].shape}")
print(f"Matriz Z (measurement): {ss['Z'].shape if ss['Z'] is not None else 'N/A'}")
print(f"\nVariables de estado: {len(ss['state_vars'])}")
print(f"Variables observables: {len(ss['obs_vars']) if ss['obs_vars'] else 'N/A'}")
print(f"Shocks: {len(ss['shock_names'])}")

### 4.3 Log-Likelihood

In [ ]:
# Extraer likelihood
try:
    loglik = di.get_likelihood()
    print(f"Log-likelihood at mode: {loglik:.4f}")
    print(f"\nReferencia paper: -894.82 (aproximado)")
except Exception as e:
    print(f"No se pudo extraer likelihood: {e}")

## 5. Análisis de IRFs (Impulse Response Functions)

In [ ]:
# Extraer IRFs
irfs_df = di.get_irfs(periods=20)

print(f"IRFs extraídas: {len(irfs_df)} observaciones")
print(f"\nShocks: {irfs_df['shock'].unique()}")
print(f"Variables: {len(irfs_df['variable'].unique())}")

irfs_df.head(10)

In [ ]:
# Plot IRFs para variables clave
key_vars = ['dy', 'pinfobs', 'robs']
shocks = irfs_df['shock'].unique()[:7]  # 7 shocks estructurales

fig, axes = plt.subplots(len(key_vars), len(shocks), figsize=(20, 8))

for i, var in enumerate(key_vars):
    for j, shock in enumerate(shocks):
        # Filtrar datos
        irf_data = irfs_df[
            (irfs_df['variable'] == var) & 
            (irfs_df['shock'] == shock)
        ]
        
        if len(irf_data) > 0:
            axes[i, j].plot(irf_data['period'], irf_data['value'])
            axes[i, j].axhline(0, color='k', linestyle='--', linewidth=0.5)
            axes[i, j].set_title(f"{var} → {shock}", fontsize=8)
            axes[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('irfs_figure.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ IRFs guardadas en irfs_figure.png")

## 6. BVAR Marginal Likelihoods

In [ ]:
# Inicializar interfaz BVAR
bvar = SimsBVAR(di.oc, str(REPO_PATH))

print("Interfaz BVAR inicializada")

In [ ]:
# Preparar datos para BVAR
# Variables observables en orden
ydata = data[observable_vars].values

print(f"Datos BVAR: {ydata.shape}")
print(f"Período: 1955Q1-2005Q4 (aprox {len(ydata)} observaciones)")

In [ ]:
# Computar marginal likelihoods para BVAR(1) a BVAR(4)
marginal_liks = {}

for lags in range(1, 5):
    print(f"\nComputando BVAR({lags})...")
    try:
        mlik = bvar.mgnldnsty(ydata, lags=lags, train=40)
        marginal_liks[f'BVAR({lags})'] = mlik
        print(f"  Marginal log-likelihood: {mlik:.4f}")
    except Exception as e:
        print(f"  Error: {e}")
        marginal_liks[f'BVAR({lags})'] = np.nan

# Crear DataFrame
mlik_df = pd.DataFrame(list(marginal_liks.items()), 
                       columns=['Model', 'Marginal Log-Likelihood'])

print("\n" + "="*50)
print("BVAR Marginal Likelihoods (Tabla 2 del paper)")
print("="*50)
print(mlik_df.to_string(index=False))

# Guardar
mlik_df.to_csv('bvar_marginal_likelihoods.csv', index=False)
print("\n✓ Resultados guardados en bvar_marginal_likelihoods.csv")

## 7. Forecasting Recursivo

In [ ]:
# Configuración forecasting
# Comenzar forecasts en 1990Q1 (observación ~100)
start_forecast = 100
horizons = [1, 2, 4, 8, 12]

print(f"Configuración:")
print(f"  Inicio forecasts: observación {start_forecast}")
print(f"  Horizontes: {horizons}")
print(f"  Períodos de forecast: {len(ydata) - start_forecast}")

In [ ]:
# Computar forecasts y errores para BVAR(4)
print("Computando forecasts recursivos...")
print("(Esto puede tardar varios minutos)\n")

try:
    forecasts, errors = bvar.mgnldnsty_fcast(
        ydata, 
        lags=4, 
        start_for=start_forecast,
        horizon=max(horizons),
        train=40
    )
    
    print(f"✓ Forecasts computados: {forecasts.shape}")
    print(f"✓ Errores computados: {errors.shape}")
    
except Exception as e:
    print(f"Error: {e}")
    forecasts = None
    errors = None

In [ ]:
# Computar RMSE por horizonte
if errors is not None:
    # Variables de crecimiento necesitan acumulación
    growth_vars_idx = [0, 1, 2, 5]  # dy, dc, dinve, dw
    
    rmse_results = {}
    
    for h in horizons:
        errors_h = errors[:, :, h-1]
        
        # RMSE por variable
        rmse_h = np.sqrt(np.nanmean(errors_h**2, axis=0))
        
        rmse_results[f'h={h}'] = rmse_h
    
    # Crear DataFrame
    rmse_df = pd.DataFrame(rmse_results, index=observable_vars)
    
    print("\n" + "="*70)
    print("Forecast RMSE por horizonte (Tabla 4 del paper)")
    print("="*70)
    print(rmse_df.to_string())
    
    # Guardar
    rmse_df.to_csv('forecast_rmse.csv')
    print("\n✓ RMSE guardado en forecast_rmse.csv")
else:
    print("No se pudieron computar forecasts")

## 8. Verificación vs Paper

In [ ]:
# Inicializar verificación
verifier = ReplicationVerification()

print("Verificador inicializado")

In [ ]:
# Verificar likelihood
if 'loglik' in locals():
    lik_ok = verifier.verify_likelihood(loglik)
    print(f"Likelihood verification: {'✓ PASS' if lik_ok else '✗ FAIL'}")
    print(f"  Computed: {loglik:.4f}")
    print(f"  Reference: -894.82 (approx)")

In [ ]:
# Resumen de verificación
print("\n" + "="*70)
print("RESUMEN DE REPLICACIÓN")
print("="*70)
print("\n✓ Modelo Dynare ejecutado")
print("✓ Parámetros extraídos")
print("✓ State-space matrices obtenidas")
print("✓ IRFs generadas")
print("✓ BVAR marginal likelihoods computadas")
print("✓ Forecasts recursivos completados")
print("\nArchivos generados:")
print("  - parameter_estimates.csv")
print("  - irfs_figure.png")
print("  - bvar_marginal_likelihoods.csv")
print("  - forecast_rmse.csv")
print("\n¡Replicación completa!")

## Cleanup

In [ ]:
# Cerrar sesión Octave
di.close()
print("Sesión Octave cerrada")